##### Firebase Firestore + Python (Week 10 – NoSQL Part 2)


##### 1. Introduction


###### Firebase Firestore + Python

In this notebook, you'll connect Python to Google Cloud Firestore using the `firebase_admin` SDK.

We'll:

- Initialize a Firestore app
- Create and populate collections (`customers`, `products`, `orders`)
- Query and update documents
- Aggregate and export results to Pandas


##### 2. Setup

###### A. Install dependencies

`pip install firebase-admin pandas`

##### B. Create a Firebase project

- Go to https://console.firebase.google.com/

- Create or open a project → “Build > Firestore Database > Start in test mode”.

- In Project Settings > Service Accounts, click
  “Generate new private key” → this downloads a JSON credentials file.


##### C. Connect in python


In [1]:
import firebase_admin
from firebase_admin import credentials,firestore
import pandas as pd
import datetime

cred = credentials.Certificate("certificate.json") # path to your key file
firebase_admin.initialize_app(cred)
db=firestore.client()
print("Connected to Firestore")

Connected to Firestore


##### 3. Define Colections


In [4]:
customers_ref = db.collection("customers")
products_ref = db.collection("products")
orders_ref = db.collection("orders")

#Firestore is document-based, so every entry is a JSON-like object.


##### 4. Insert sample data


In [5]:
customers_data = [
    {"id":1,"first_name":"Daniel","last_name":"Masila","email":"daniel@example.com","city":"Nairobi","region":"Central"},
    {"id":2,"first_name":"Jane","last_name":"Mwangi","email":"jane@example.com","city":"Nakuru","region":"Rift Valley"},
    {"id":3,"first_name":"John","last_name":"Otieno","email":"john@example.com","city":"Kisumu","region":"Western"},
]

for c in customers_data:
    customers_ref.document(str(c['id'])).set(c)
    
products_data = [
    {"id":1,"name":"Wireless Mouse","category":"Accessories","price":1200},
    {"id":2,"name":"Laptop Stand","category":"Accessories","price":3500},
    {"id":3,"name":"HDMI Cable","category":"Cables","price":800},
    {"id":4,"name":"Monitor 24 inch","category":"Electronics","price":16000},
]
for p in products_data:
    products_ref.document(str(p["id"])).set(p)

orders_data = [
    {"id":1,"customer_id":1,"product_id":1,"order_date":datetime.date(2025,10,1).isoformat(),"quantity":2,"status":"Delivered"},
    {"id":2,"customer_id":1,"product_id":2,"order_date":datetime.date(2025,10,3).isoformat(),"quantity":1,"status":"Delivered"},
    {"id":3,"customer_id":2,"product_id":4,"order_date":datetime.date(2025,10,5).isoformat(),"quantity":1,"status":"Shipped"},
]
for o in orders_data:
    orders_ref.document(str(o["id"])).set(o)

print("data inserted into firestore")

data inserted into firestore


##### 5. Query Documents


In [6]:
for doc in customers_ref.stream():
    print(doc.to_dict())

{'region': 'Central', 'email': 'daniel@example.com', 'last_name': 'Masila', 'first_name': 'Daniel', 'id': 1, 'city': 'Nairobi'}
{'region': 'Rift Valley', 'email': 'jane@example.com', 'last_name': 'Mwangi', 'first_name': 'Jane', 'id': 2, 'city': 'Nakuru'}
{'region': 'Western', 'email': 'john@example.com', 'last_name': 'Otieno', 'first_name': 'John', 'id': 3, 'city': 'Kisumu'}


In [8]:
## filter example
for doc in orders_ref.where("status","==","Delivered").stream():
    print(doc.to_dict())

d:\PYTHON END TO END\virtual_env\Lib\site-packages\google\cloud\firestore_v1\base_collection.py:304: UserWarning: Detected filter using positional arguments. Prefer using the 'filter' keyword argument instead.
  return query.where(field_path, op_string, value)


{'status': 'Delivered', 'quantity': 2, 'order_date': '2025-10-01', 'product_id': 1, 'id': 1, 'customer_id': 1}
{'status': 'Delivered', 'quantity': 1, 'order_date': '2025-10-03', 'product_id': 2, 'id': 2, 'customer_id': 1}


##### 6. Manual Join style Query (in Python)


In [9]:
#NB :Firestore has no server-side joins, but you can combine data in Python:

orders = [o.to_dict() for o in orders_ref.stream()]
customers = {c.to_dict()['id']: c.to_dict() for c in customers_ref.stream()}
products = {p.to_dict()['id']: p.to_dict() for p in products_ref.stream()}

records = []
for o in orders:
    cust = customers[o['customer_id']]
    prod =products[o['product_id']]
    records.append({
        "customer":cust['first_name'],
        "region":cust["region"],
        "product":prod['name'],
        "category":prod['category'],
        "total_value":o["quantity"] * prod["price"]
    })

df = pd.DataFrame(records)
df.head()

,customer,region,product,category,total_value
0,Daniel,Central,Wireless Mouse,Accessories,2400
1,Daniel,Central,Laptop Stand,Accessories,3500
2,Jane,Rift Valley,Monitor 24 inch,Electronics,16000


##### 7. aggregation in pandas


In [10]:
agg = (
    df.groupby("region")["total_value"]
    .sum()
    .reset_index()
    .sort_values("total_value",ascending=False)
)

agg

,region,total_value
1,Rift Valley,16000
0,Central,5900


##### 8. Visulization


In [13]:
import plotly.express as px
px.bar(agg,x='region',y='total_value',title="Sales By Region")

##### 9. Update and delete


In [14]:
# update
orders_ref.document("3").update({"status":"Delivered"})

# delete
products_ref.document("4").delete()

DatetimeWithNanoseconds(2025, 10, 15, 10, 25, 25, 242269, tzinfo=datetime.timezone.utc)

##### 11. Summary


we learned to:

- Connect to Google Firestore using the Firebase Admin SDK
- Insert and query nested JSON-like documents
- Merge data across collections with client-side logic
- Aggregate and visualize results with Pandas

Next up (optional): compare **Firestore vs MongoDB vs SQL** for architecture and performance trade-offs.


##### Quick Comparison

| **Feature**        | **Firestore**                    | **MongoDB**               |
| ------------------ | -------------------------------- | ------------------------- |
| **Hosting**        | Google Cloud                     | Self-hosted / Atlas Cloud |
| **Structure**      | Collections → Documents → Fields | Same hierarchy            |
| **Schema**         | Flexible                         | Flexible                  |
| **Query Language** | Firestore API (MQL-like)         | MongoDB Query Language    |
| **Transactions**   | Multi-document supported         | Supported                 |
| **Ideal Use**      | Cloud apps, mobile sync          | Local / server-based apps |
